Maciek Małecki
# Analisys of text and visualisation on map

Download the novel “Around the world in 80 days” by J. Verne from Project
Gutenberg website (link: http://www.gutenberg.org/files/103/103-h/103-h.htm). Find all cities visited by Phileas Fogg, the protagonist of
the novel. Draw the path of his journey on a world map.


### Library

In [0]:
!pip install bs4
!pip install lxml
!pip install geotext
!pip install folium
!pip install geopy

  Running setup.py bdist_wheel for bs4 ... - done
  Stored in directory: /root/.cache/pip/wheels/a0/b0/b2/4f80b9456b87abedbc0bf2d52235414c3467d8889be38dd472
Successfully built bs4
    100% |████████████████████████████████| 5.8MB 4.2MB/s 
    100% |████████████████████████████████| 2.0MB 13.7MB/s 
    100% |████████████████████████████████| 81kB 2.2MB/s 
    100% |████████████████████████████████| 92kB 2.5MB/s 
  Running setup.py bdist_wheel for geographiclib ... - done
  Stored in directory: /root/.cache/pip/wheels/99/45/d1/14954797e2a976083182c2e7da9b4e924509e59b6e5c661061
Successfully built geographiclib


In [0]:
import requests
from  bs4 import BeautifulSoup
import lxml
import re
import matplotlib.pyplot as plt
from geotext import GeoText
import numpy as np
import folium
import time
from geopy.geocoders import Nominatim
from operator import itemgetter

## Definition


In [0]:
def import_chapter(index_url):
    r = requests.get(index_url)
    soup = BeautifulSoup(r.content,"lxml")
    data=soup.find_all('p')
    lenght = len(data)
#     list_h = list_head
    return data, lenght

def cleaning(words):
    words = words.replace("Bombay", "Mumbai")
    words = words.replace("Calcutta", "Kolkata")
    words = words.replace("Hong Kong", "Victoria")
    words = words.replace("Stuart","")
    words = words.replace('"',' ')
    words = words.replace("Forster","")
    words = words.replace("Wilson","")
    words = words.replace("Reno","")
    return words

def remove_empty_place(city_around_the_world):
    city = []
    for i in range(len(city_around_the_world)):
        for j in range(len(city_around_the_world[i][0])):
            city.append(city_around_the_world[i][0][j])
    return city

def choose_cities(data):
    keys_words = ['beetween','from','to','was','arrived','Left','left','reached','visited', 'departed',' abandoned','stayed','stopped','arrival', 'Reached']
    city_around_the_world = []
    for paragraph in data:
        part_text = str(paragraph)
        part_text = part_text.replace("\r\n"," ")
        part_text = part_text.replace(","," ")
        sentence = part_text.split(".")
        city_paragraph = []
        
        for words in sentence:
            words = cleaning(words)
            wordssss = words.split(" ")
            correct_words = [x for x in wordssss if x != '<p>' and x != '' and x != '</p>']
            places = GeoText(words)
            names_cities = places.cities
            city_sentens = []
            
            if len(names_cities) != 0:
                for city in names_cities:
#                     print(correct_words)
#                     print(city)
                    if city in correct_words:
                        position_city = correct_words.index(city)
                        for check_keys_words in keys_words:
                            if check_keys_words in correct_words:
                                position_keys_word = correct_words.index(check_keys_words)
                                if position_city - position_keys_word < 15 and position_city - position_keys_word >0 :
                                    city_sentens.append(city)
                city_sentens = list(set(city_sentens))
            city_paragraph.append(city_sentens)
        if len(city_paragraph)!=0:
            city_around_the_world.append(city_paragraph)        
    city = remove_empty_place(city_around_the_world)
    return city, city_around_the_world
#     print(len(list(set(city))))    
    # list_of_city = list(set(city))
#     print(len(city))

def remove_duplication(date):
    city_without_duplication = []
    for i in city:
        if i not in city_without_duplication:
            city_without_duplication.append(i)
    city_without_duplication.append('London')
    return city_without_duplication  
    
    
def localisation(cities):
    local = []
    for city in cities:
        c = geocoder.google(city)
        local.append((c.latlang[0], c.latlang[1]))
    

    
def localization(cities):
    geolocator = Nominatim()
    positions = dict()
    for c in cities:
        while True:
            try:
                position = geolocator.geocode(c)
            except:
                time.sleep(0.1)
                continue
            break
        if position:
            location = [position.latitude, position.longitude]
            positions.update({c : location})
            del position
        else:
            print("Could not get position for {}".format(c))
    positions = sorted(list(positions.values()), key=lambda x: x[1])        
    return positions
    
    

def show_map(positions):
    mapa = folium.Map(location=[0, 0], zoom_start=4)
    locations = []
    for location in positions:
#         mapa.add_child(folium.Marker(location=location,popup=folium.Popup(city),icon=folium.Icon(color='orange',icon='ok')))
        #add a markers
        folium.Marker(location).add_to(mapa)
        locations.append(location)
    folium.PolyLine(locations, color="red", weight=2.5, opacity=1).add_to(mapa)
    return mapa 

In [13]:
index_url = 'http://www.gutenberg.org/files/103/103-h/103-h.htm'
data, lenght = import_chapter(index_url)
city,cc = choose_cities(data)
city = remove_duplication(city)
citypoint = city.index("Victoria")
city[citypoint] = "Hong Kong"
position = localization(city)
# position = sorted(list(position.values()), key=lambda x: x[1])
show_map(position)
# print(len(city))
# print(city)
# print(len(city))
# print(cc)

/usr/local/lib/python3.6/dist-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


In [0]:
#City - 
# Hong Kong, -> Xiānggǎng 
# Bombaj, -> Mumbai
# Singapur,
# Calcuta -> Kolkata
places = GeoText("Turin is a great city")
places.cities

['Turin']

In [8]:
print(len(city))
print(city)
# print(len(city))
print(cc)

17
['London', 'Paris', 'Suez', 'Mumbai', 'Kolkata', 'Hong Kong', 'Buxar', 'Patna', 'Yokohama', 'Shanghai', 'Oakland', 'Missouri', 'Ogden', 'Omaha', 'Kearney', 'Liverpool', 'London']
[[[], [], [], [], [], []], [[], [], [], [], [], []], [[], []], [[], []], [[], [], []], [[], [], [], [], [], [], [], []], [[], [], [], []], [['London'], [], [], [], [], [], [], []], [[], [], [], [], [], [], [], [], []], [[], []], [[], [], [], []], [[], [], [], []], [[], []], [[], []], [[], []], [[]], [[], [], [], ['Paris'], [], [], []], [[], [], [], []], [[], []], [[]], [[], []], [[], [], []], [[]], [[], [], [], [], [], []], [[], []], [[], [], [], []], [[]], [[], []], [[], [], [], [], [], [], [], [], []], [[], [], [], []], [[], []], [['Paris'], [], [], [], [], []], [[], [], [], [], [], [], [], [], [], [], []], [[], [], [], [], [], [], [], []], [[], [], [], [], [], []], [[], [], [], [], [], [], [], []], [[], [], []], [[], [], [], [], [], [], [], [], [], [], [], []], [[]], [[], []], [[], [], []], [[], []], [[]

### Solution

In [0]:
index_url = 'http://www.gutenberg.org/files/103/103-h/103-h.htm'
data, lenght = import_chapter(index_url)
keys_words = ['was','arrived','left','reached','visited', 'departed',' abandoned','stayed','stopped','arrival']
city_around_the_world = []
print(len(data))
for paragraph in data[:1]:
    part_text = str(paragraph)
    part_text = part_text.replace("\r\n"," ")
    part_text = part_text.replace(",","")
    sentence = part_text.split(".")
    city_paragraph = []
    for words in sentence:
        wordssss = words.split(" ")
        print(wordssss)
        correct_words = [x for x in wordssss if x != '<p>' and x != '' and x != '</p>']
#         print(correct_words)
        places = GeoText(words)
        names_cities = places.cities
        city_sentens = []
        if len(names_cities) != 0:
            for city in names_cities:
                position_city = correct_words.index(city)
        #         position_city_in_sentens = correct_words.index(city)
                for check_keys_words in keys_words:
#                     print(correct_words)
                    if check_keys_words in correct_words:
                        print(check_keys_words)
                        position_keys_word = correct_words.index(check_keys_words)
                        if np.abs(position_city - position_keys_word) < 3:
                            city_sentens.append(city)
            city_sentens = list(set(city_sentens))
        city_paragraph.append(city_sentens)
    if len(city_paragraph)!=0:
        city_around_the_world.append(city_paragraph)

# city_around_the_world.remove([[]])
city = []
for i in range(len(city_around_the_world)):
    for j in range(len(city_around_the_world[i][0])):
        city.append(city_around_the_world[i][0][j] )
    
print(len(list(set(city))))    
# list_of_city = list(set(city))
print(len(city))
city_without_duplication = []
for i in city:
    if i not in city_without_duplication:
        city_without_duplication.append(i)
city_without_duplication.append('London')
print(len(city_without_duplication))
city_without_duplication

#City - Hong Kong, Bombaj, Singapur, Calcuta

1604
['<p>', 'Mr']
['', 'Phileas', 'Fogg', 'lived', 'in', '1872', 'at', 'No']
['', '7', 'Saville', 'Row', 'Burlington', 'Gardens', 'the', 'house', 'in', 'which', 'Sheridan', 'died', 'in', '1814']
['', '', 'He', 'was', 'one', 'of', 'the', 'most', 'noticeable', 'members', 'of', 'the', 'Reform', 'Club', 'though', 'he', 'seemed', 'always', 'to', 'avoid', 'attracting', 'attention;', 'an', 'enigmatical', 'personage', 'about', 'whom', 'little', 'was', 'known', 'except', 'that', 'he', 'was', 'a', 'polished', 'man', 'of', 'the', 'world']
['', '', 'People', 'said', 'that', 'he', 'resembled', 'Byron—at', 'least', 'that', 'his', 'head', 'was', 'Byronic;', 'but', 'he', 'was', 'a', 'bearded', 'tranquil', 'Byron', 'who', 'might', 'live', 'on', 'a', 'thousand', 'years', 'without', 'growing', 'old']
['', '</p>']
0
0
1


['London']